In [1]:
import os
import requests

def download_tarball(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as f:
        f.write(response.content)

def main():
    # Base URL where the tarballs are located
    base_url = "https://tess.mit.edu/public/tesstransients/pages/"
    
    # Range of tarball numbers
    start_num = 1
    end_num = 66
    
    # Local directory where you want to save the tarballs
    local_directory = r"C:\Users\nic51\TESS RESEARCH\Tess_tarballs"
    
    # Create the directory if it does not exist
    if not os.path.exists(local_directory):
        os.makedirs(local_directory)
    
    # Download each tarball file
    for i in range(start_num, end_num + 1):
        # Pad the number with leading zeros if necessary (e.g., s01, s02, ..., s66)
        tarball_num = str(i).zfill(2)
        # Construct the relative URL for the current tarball
        relative_url = f"../lc_bulk/s{tarball_num}.tgz"
        # Construct the full URL by joining the base URL and the relative URL
        full_url = base_url + relative_url
        # Extract the filename from the relative URL
        filename = os.path.basename(relative_url)
        # Construct the local save path
        local_save_path = os.path.join(local_directory, filename)
        # Download the tarball
        print(f"Downloading {filename} from {full_url}...")
        download_tarball(full_url, local_save_path)
        print(f"Downloaded {filename} to {local_save_path}")

if __name__ == "__main__":
    main()


Downloaded s01.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s01.tgz
Downloaded s02.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s02.tgz
Downloaded s03.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s03.tgz
Downloaded s04.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s04.tgz
Downloaded s05.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s05.tgz
Downloaded s06.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s06.tgz
Downloaded s07.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s07.tgz
Downloaded s08.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s08.tgz
Downloaded s09.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s09.tgz
Downloaded s10.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s10.tgz
Downloaded s11.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s11.tgz
Downloaded s12.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s12.tgz
Downloaded s13.tgz to C:\Users\nic51\TESS RESEARCH\Tess_tarballs\s13.tgz
Downloaded s14.tgz to C:\Users\nic51\TESS RESEARCH\

In [2]:
#This extracts all .tgz files and deletes all .png files, all while adding the .txt extension :)
import os
import tarfile

def extract_tarball(tgz_file, extraction_dir, delete_png=False, add_txt_extension=False):
    try:
        with tarfile.open(tgz_file, 'r:gz') as tar:
            tar.extractall(path=extraction_dir)
            if delete_png:
                # Delete .png files
                for member in tar.getmembers():
                    if member.name.endswith('.png'):
                        os.remove(os.path.join(extraction_dir, member.name))
            if add_txt_extension:
                # Add .txt extension to each kept file
                for member in tar.getmembers():
                    if not member.name.endswith('.png'):
                        new_name = os.path.splitext(member.name)[0] + '.txt'
                        os.rename(os.path.join(extraction_dir, member.name), os.path.join(extraction_dir, new_name))
        print(f"Extracted {os.path.basename(tgz_file)} to {extraction_dir}")
    except tarfile.ReadError as e:
        print(f"Error extracting {os.path.basename(tgz_file)}:", e)

def main():
    # Local directory where the .tgz files are located
    local_directory = r"C:\Users\nic51\TESS RESEARCH\Tess_tarballs"
    
    # Directory where you want to extract the .tgz files
    extraction_dir = r"C:\Users\nic51\TESS RESEARCH\Extracted_tarballs"
    
    # Create the extraction directory if it does not exist
    if not os.path.exists(extraction_dir):
        os.makedirs(extraction_dir)
    
    # Get a list of .tgz files in the local directory
    tgz_files = [f for f in os.listdir(local_directory) if f.endswith('.tgz')]
    
    # Extract each .tgz file
    for tgz_file in tgz_files:
        # Construct the full path of the .tgz file
        tgz_file_path = os.path.join(local_directory, tgz_file)
        # Extract the .tgz file and optionally delete .png files and add .txt extension
        extract_tarball(tgz_file_path, extraction_dir, delete_png=True, add_txt_extension=True)

if __name__ == "__main__":
    main()

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\nic51\\TESS RESEARCH\\Extracted_tarballs\\sector01/cam1_ccd1/lc_discovery/lc_2018evo_cleaned' -> 'C:\\Users\\nic51\\TESS RESEARCH\\Extracted_tarballs\\sector01/cam1_ccd1/lc_discovery/lc_2018evo_cleaned.txt'

In [4]:
#This code takes all figures generated in each sector notebook and saves them as jpeg files
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from PIL import Image
import base64
import io

# Function to execute a notebook
def execute_notebook(notebook_path):
    with open(notebook_path) as f:
        nb = nbformat.read(f, as_version=4)

    # Execute the notebook
    ep = ExecutePreprocessor(timeout=600, kernel_name='python3')
    ep.preprocess(nb)

# Function to extract and save plots
def save_plots(notebook_path, output_dir):
    with open(notebook_path) as f:
        nb = nbformat.read(f, as_version=4)

    # Execute the notebook
    execute_notebook(notebook_path)

    # Extract plots from the executed notebook
    for cell in nb.cells:
        if 'outputs' in cell and cell.cell_type == 'code':
            for output in cell.outputs:
                if output.output_type == 'display_data' and 'image/png' in output.data:
                    image_data = output.data['image/png']
                    # Decode base64 encoded image data
                    image_bytes = base64.b64decode(image_data)
                    # Save the plot as a JPEG image file
                    plot_name = f"plot_{cell.execution_count}.jpeg"  # Change the file extension
                    plot_path = os.path.join(output_dir, plot_name)
                    with Image.open(io.BytesIO(image_bytes)) as img:
                        img.convert('RGB').save(plot_path, format='JPEG')  # Save as JPEG

# Function to process notebooks for sectors
def process_sectors():
    for sector in range(1, 67):
        sector_notebook_path = rf'C:\Users\nic51\TESS RESEARCH\Interesting_curves\sector_{sector}.ipynb'
        sector_output_dir = rf'C:\Users\nic51\TESS RESEARCH\Interesting_curves\sector{sector}_interestingcurves'
        os.makedirs(sector_output_dir, exist_ok=True)  # Create directory if it doesn't exist
        save_plots(sector_notebook_path, sector_output_dir)

process_sectors()


KeyboardInterrupt: 

In [33]:
import os
from pptx import Presentation
from pptx.util import Inches
from PIL import Image
import tempfile

def resize_image(image_path, max_width, max_height):
    image = Image.open(image_path)
    width, height = image.size
    aspect_ratio = width / height

    if width > max_width or height > max_height:
        if aspect_ratio > 1:
            new_width = max_width
            new_height = max_width / aspect_ratio
        else:
            new_width = max_height * aspect_ratio
            new_height = max_height

        image = image.resize((int(new_width), int(new_height)), Image.ANTIALIAS)

    return image

def create_ppt_from_images(root_folder, output_pptx):
    prs = Presentation()
    max_width = Inches(8)
    max_height = Inches(6)

    for folder in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder)
        if os.path.isdir(folder_path):
            # Add title slide for each folder
            title_slide_layout = prs.slide_layouts[0]  # Index 0 corresponds to the title slide layout
            title_slide = prs.slides.add_slide(title_slide_layout)
            title = title_slide.shapes.title
            title.text = folder

            for file_name in os.listdir(folder_path):
                if file_name.endswith(".jpg") or file_name.endswith(".jpeg"):
                    file_path = os.path.join(folder_path, file_name)
                    resized_image = resize_image(file_path, max_width, max_height)
                    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
                        temp_file_path = temp_file.name
                        resized_image.save(temp_file_path, format="JPEG")
                        slide_layout = prs.slide_layouts[6]
                        slide = prs.slides.add_slide(slide_layout)
                        left = top = Inches(1)
                        slide.shapes.add_picture(temp_file_path, left, top, width=max_width, height=max_height)

    prs.save(output_pptx)
    print("PowerPoint presentation created successfully!")

root_folder = r"C:\Users\nic51\TESS RESEARCH\Interesting_curves"
output_pptx = "output2.pptx"

create_ppt_from_images(root_folder, output_pptx)


PowerPoint presentation created successfully!
